#  OCR (Optical Character Recognition) - Implantação
## Utilização das bibliotecas [opencv](https://opencv.org/) e  [Tesseract OCR](https://tesseract-ocr.github.io/) para o reconhecimento de texto em imagens e da biblioteca [JiWER](https://github.com/jitsi/jiwer) para cálculo de mérticas de perfomance

*   Mais detlalhes sobre  o funcionamento dos algorítimos e das línguas nos quais o mesmo podem são utilizados são encontrados na [Tesseract documentation](https://tesseract-ocr.github.io/tessdoc/Data-Files)

*   Caso seja passado um arquivo .xlsx com as strings de target pode visualizar a perfonrmance do algorítimo

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py
import joblib
import nltk
import pandas as pd
import numpy as np
import base64
import cv2
import logging
from ocr import Class_Pytesseract_OCR


class Model:
    
    def __init__(self):
        self.loaded = False
    
    def load(self):
        artifacts = joblib.load("/tmp/data/ocr.joblib")
        self.hyperparams = artifacts["hyperparams"]
        self.model_parameters = artifacts["model_parameters"]
        self.return_formats = artifacts["return_formats"]
        
        self.model = Class_Pytesseract_OCR(self.hyperparams, self.model_parameters, self.return_formats)
        # Health check validation
        random_img = np.random.randint(0, 255, size=(100, 100, 3), dtype=np.uint8)
        _, _ = self.model.predict(random_img, "Deployment")
        
        
        
        self.loaded = True 

        
    def class_names(self):
        if self.return_formats['bbox_return'] == "image":
            return ['image', 'text']
        return ['x_min', 'y_min', 'x_max', 'y_max', 'text']
    
    def format_result(self, boxes, text):
        words = text.split(' ')
        words.remove('')
        res = []
        for i, box in enumerate(boxes):
            box = list(map(float, box))
            res.append(
                np.array(box + [words[i]])
            )
        """
        for i in range(len(boxes)):
            
            bbox = list(map(float, boxes[i]))
            bbox.extend([text[i]])
            bbox = np.array(bbox)
            res.append(bbox)
        """

        
        return res
    
    def predict(self, X, feature_names, meta=None):
        # First time load model
        if not self.loaded:
            self.load()
            
        # Check if data is a bytes
        if isinstance(X, bytes):
            im_bytes = X # Get image bytes
        
        # If not, should be a list or ndarray
        else:
            # Garantee is a ndarray
            X = np.array(X)
            
            # Seek for extra dimension
            if len(X.shape) == 2:
                im_bytes = X[0,0] # Get image bytes
            
            else:
                im_bytes = X[0] # Get image bytes
        
        # Preprocess img bytes to img_arr
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        frame = np.array(img).astype(np.uint8)
        
        bboxes_or_image, text = self.model.predict(frame, "Deployment", self.return_formats)
        
        if self.return_formats['bbox_return'] == "np_array":
            result = self.format_result(bboxes_or_image, text)
        else:
            im_arr = np.frombuffer(bboxes_or_image, dtype=np.uint8)
            img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            frame_with_text = np.array(img).astype(np.uint8)
            result = frame_with_text
        
        ### DEBUG ###
        logging.error('--- DBG ---')
        logging.error('result: %s', result)
        logging.error('type: %s', type(result))
        logging.error('--- DBG ---')
        #############
        
        return np.array(result)

Overwriting Model.py


In [2]:
# Testing
"""
from Model import Model
import time
import base64

if __name__ == '__main__':

    print('>> Loading model...', end=' ')
    start = time.time()
    model = Model()
    print(f'{time.time() - start:.2f}s')

    print('>> Loading image...', end=' ')
    start = time.time()
    encoded = b'...'
    decoded = base64.b64decode(encoded)
    print(f'{time.time() - start:.2f}s')

    print('>> Predicting...', end=' ')
    start = time.time()
    model.predict(decoded, feature_names=[])
    print(f'{time.time() - start:.2f}s')
"""



>> Loading model... 0.00s
>> Loading image... 0.00s
>> Predicting... 

ERROR:root:--- DBG ---
ERROR:root:result: [array([[[218, 218, 254],
        [218, 218, 254],
        [218, 218, 254],
        ...,
        [218, 218, 254],
        [218, 218, 254],
        [218, 218, 254]],

       [[218, 218, 254],
        [218, 218, 254],
        [218, 218, 254],
        ...,
        [218, 218, 254],
        [218, 218, 254],
        [218, 218, 254]],

       [[218, 218, 254],
        [218, 218, 254],
        [218, 218, 254],
        ...,
        [218, 218, 254],
        [218, 218, 254],
        [218, 218, 254]],

       ...,

       [[218, 218, 254],
        [218, 218, 254],
        [218, 218, 254],
        ...,
        [218, 218, 254],
        [218, 218, 254],
        [218, 218, 254]],

       [[218, 218, 254],
        [218, 218, 254],
        [218, 218, 254],
        ...,
        [218, 218, 254],
        [218, 218, 254],
        [218, 218, 254]],

       [[218, 218, 254],
        [218, 218, 254],
        [218, 218, 254],
        ...,
        [218, 218, 254],
      

0.71s


/home/jovyan/tasks/OCR (Optical Character Recognition)/Model.py:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(result)
